In [1]:
import sys
sys.path.append('libsigma/')
import os
import read_and_write as rw
import geopandas as gpd
import numpy as np

In [1]:
def code_vege(shapefile_path_vege,shapefile_path_emprise,output_path):
   
    # Charger le fichier shapefile
    gdf_vegetation = gpd.read_file(shapefile_path_vege)
    gdf_emprise = gpd.read_file(shapefile_path_emprise)
    
    # Ajouter les champs "nom" et "code" avec des valeurs par défaut
    gdf_vegetation['nom'] = None
    gdf_vegetation['code'] = None
    
   
    # Création d'un dictionnaire pour mapper les valeurs de 'TFV' aux colonnes 'nom' et 'code'
    mapping = {
        "FF1-49-49": ("Autres feuillus", "11"),
        "FF1-10-10": ("Autres feuillus", "11"),
        "FF1-09-09": ("Autres feuillus", "11"),
        "FF1G01-01": ("Chêne", "12"),
        "FF1-14-14": ("Robinier", "13"),
        "FP": ("Peupleraie", "14"),
        "FF1-00-00": ("Mélange de feuillus", "15"), 
        "FF1-00": ("Feuillus en îlots", "16"),
        "FF2-91-91": ("Autres conifères autre que pin", "21"),
        "FF2-63-63": ("Autres conifères autre que pin", "21"),
        "FF2G61-61": ("Autres conifères autre que pin", "21"),
        "FF2-90-90": ("Autres conifères autre que pin", "21"),
        "FF2-81-81": ("Autres Pin", "22"),
        "FF2-52-52": ("Autres Pin", "22"),
        "FF2-80-80": ("Autres Pin", "22"),
        "FF2-64-64": ("Douglas", "23"),
        "FF2G53-53": ("Pin laricio ou pin noir", "24"),
        "FF2-51-51": ("Pin maritime", "25"),
        "FF2-00-00": ("Mélange conifères", "26"),
        "FF2-00": ("Conifères en îlots", "27"),
        "FF32": ("Mélange de conifères prépondérants et feuillus", "28"),
        "FF31": ("Mélange de feuillus prépondérants et conifères", "29"),
    }
    
    # Application des valeurs du mapping
    gdf_vegetation['nom'] = gdf_vegetation['CODE_TFV'].map(lambda x: mapping[x][0] if x in mapping else None)
    gdf_vegetation['code'] = gdf_vegetation['CODE_TFV'].map(lambda x: mapping[x][1] if x in mapping else None)
    
    gdf_vegetation = gdf_vegetation.dropna(subset=['code'])
    
    
    vege_emprise = gpd.clip(gdf_vegetation, gdf_emprise)
    # Sauvegarder les modifications dans un nouveau fichier (ou remplacer l'original)
    
    vege_emprise.to_file(output_path)

In [3]:
shapefile_path_vege = 'data/project/FORMATION_VEGETALE.shp'
shapefile_path_emprise = 'data/project/emprise_etude.shp'
output_path = 'Sample_BD_foret_T31TCJ.shp'
code_vege(shapefile_path_vege,shapefile_path_emprise,output_path)

In [2]:
def rasterize(in_vector,out_image,field_name,sptial_resolution):

  # define command pattern to fill with paremeters
  cmd_pattern = ("gdal_rasterize -a {field_name} "
                 "-tr {sptial_resolution} {sptial_resolution} "
                 "-ot Byte -of GTiff "
                 "-a_nodata 0 "
                 "{in_vector} {out_image}")
  
  # fill the string with the parameter thanks to format function
  cmd = cmd_pattern.format(in_vector=in_vector, out_image=out_image, field_name=field_name,
                           sptial_resolution=sptial_resolution)
  
  # execute the command in the terminal
  print(cmd)
  os.system(cmd)

In [5]:
in_vector = 'Sample_BD_foret_T31TCJ.shp'
out_image = 'raster.tif'
field_name = 'code'
sptial_resolution = 10.0
rasterize(in_vector,out_image,field_name,sptial_resolution)

gdal_rasterize -a code -tr 10.0 10.0 -ot Byte -of GTiff -a_nodata 0 Sample_BD_foret_T31TCJ.shp raster.tif


In [6]:
def masque(input_image,output_masque):
    data_set = rw.open_image(input_image)
    img = rw.load_img_as_array(input_image)
    
    
    # do some processingwith numpy: build a mask
    nb_ligne, nb_col, nb_band = rw.get_image_dimension(data_set)
    masque_foret = img != 0
    
    # write it
    rw.write_image(output_masque, masque_foret, data_set=data_set,
                gdal_dtype=data_type_match['uint8'])

In [7]:
data_type_match = {'uint8': gdal.GDT_Byte,
                   'uint16': gdal.GDT_UInt16,
                   'uint32': gdal.GDT_UInt32,
                   'int16': gdal.GDT_Int16,
                   'int32': gdal.GDT_Int32,
                   'float32': gdal.GDT_Float32,
                   'float64': gdal.GDT_Float64}
input_image = 'raster.tif'
output_masque = 'masque_foret.tif'
masque(input_image,output_masque)

C:\Users\pabra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
input_emprise = 'data/project/emprise_etude.shp'
gdf = gpd.read_file(input_emprise)
gdf['rasterize'] = 1.0
field_name = 'rasterize'
output_emprise = 'data/project/emprise.tif'
sptial_resolution = 10.0
rasterize(input_emprise,output_emprise,field_name,sptial_resolution)

gdal_rasterize -a rasterize -tr 10.0 10.0 -ot Byte -of GTiff -a_nodata 0 data/project/emprise_etude.shp data/project/emprise.tif


In [3]:
def reproject_raster(minx,miny,maxx,maxy,input_raster, output_raster, src_epsg, dst_epsg):
    # Define the command pattern to fill with parameters
    cmd_pattern = ("gdalwarp -s_srs EPSG:{src_epsg} "
                    "-t_srs EPSG:{dst_epsg} "
                    "-te {minx} {miny} {maxx} {maxy} -te_srs EPSG:{dst_epsg} "
                    "-tr 10 10 "
                    "-r near "
                    "-of GTiff "
                    "{input_raster} {output_raster}")
    
    # Fill the string with the parameters using format function
    cmd = cmd_pattern.format(minx=minx,miny=miny,maxx=maxx,maxy=maxy,input_raster=input_raster, output_raster=output_raster,
                             src_epsg=src_epsg, dst_epsg=dst_epsg)
    
    # Execute the command in the terminal
    print(cmd)
    os.system(cmd)

In [15]:
def preparation(releves,bands,emprise,liste_img):
    suffixe = '.tif'
    gdf = gpd.read_file(emprise)
    gdf['rasterize'] = 1.0
    field_name = 'rasterize'
    output_emprise = 'data/project/emprise.tif'
    sptial_resolution = 10.0
    rasterize(emprise,output_emprise,field_name,sptial_resolution)
    dataset = rw.open_image(output_emprise)
    geotransform = dataset.GetGeoTransform()
    minx = geotransform[0]  # Origine X
    maxy = geotransform[3]  # Origine Y
    maxx = minx + geotransform[1] * dataset.RasterXSize  
    miny = maxy + geotransform[5] * dataset.RasterYSize  
    src_epsg = 32631  # EPSG 32632 (UTM zone 32N)
    dst_epsg = 2154   # EPSG 2154 (RGF93 / Lambert-93)
    for r in releves:
        for B in bands:
            band_name = f'data/images/SENTINEL2{r}{B}'
            band_file = f'{band_name}{suffixe}'  # Nom du fichier
            output_filename = f"{band_name}_10_2154{suffixe}"
            reproject_raster(minx,miny,maxx,maxy,band_file,output_filename,src_epsg,dst_epsg)
            liste_img.append(rw.load_img_as_array(output_filename))

In [16]:
emprise = 'data/project/emprise_etude.shp'
bands = [2, 3, 4, 5, 6, 7, 8, '8A', 11, 12]
releves = ['B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B','B_20220405-105855-542_L2A_T31TCJ_C_V3-0_FRE_B','B_20220803-105903-336_L2A_T31TCJ_C_V3-0_FRE_B','B_20221111-105858-090_L2A_T31TCJ_C_V3-1_FRE_B','A_20221116-105900-865_L2A_T31TCJ_C_V3-1_FRE_B','B_20230209-105857-157_L2A_T31TCJ_C_V3-1_FRE_B']
img_all_band = []

preparation(releves,bands,emprise,img_all_band)

gdal_rasterize -a rasterize -tr 10.0 10.0 -ot Byte -of GTiff -a_nodata 0 data/project/emprise_etude.shp data/project/emprise.tif
gdalwarp -s_srs EPSG:32631 -t_srs EPSG:2154 -te 501122.9697 6240649.0236 609762.9697 6314469.0236 -te_srs EPSG:2154 -tr 10 10 -r near -of GTiff data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2_10_2154.tif
gdalwarp -s_srs EPSG:32631 -t_srs EPSG:2154 -te 501122.9697 6240649.0236 609762.9697 6314469.0236 -te_srs EPSG:2154 -tr 10 10 -r near -of GTiff data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B3.tif data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B3_10_2154.tif
gdalwarp -s_srs EPSG:32631 -t_srs EPSG:2154 -te 501122.9697 6240649.0236 609762.9697 6314469.0236 -te_srs EPSG:2154 -tr 10 10 -r near -of GTiff data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B4.tif data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FR

In [17]:
output_img = 'Serie_temp_S2_allbands.tif'
image_filename = 'data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2_10_2154.tif'
img = np.concatenate(img_all_band,axis=-1)
data_set = rw.open_image(image_filename)
rw.write_image(output_img, img, data_set=data_set)